# **RUN**

Un run es:

- un algoritmo (flow)

- con hiperparámetros

- entrenado y evaluado en una task

In [1]:
import openml
import pandas as pd
from openml.tasks import TaskType

run = openml.runs.get_run(10595819)
print(run.evaluations)


{'area_under_roc_curve': 0.9944666666666667, 'average_cost': 0.0, 'f_measure': 0.9533286661999534, 'kappa': 0.93, 'kb_relative_information_score': 0.9289119924022927, 'mean_absolute_error': 0.03688888888888889, 'mean_prior_absolute_error': 0.44444444444444337, 'weighted_recall': 0.9533333333333334, 'number_of_instances': 150.0, 'precision': 0.9534480458850206, 'predictive_accuracy': 0.9533333333333333, 'prior_entropy': 1.5849625007211599, 'relative_absolute_error': 0.0830000000000002, 'root_mean_prior_squared_error': 0.4714045207910311, 'root_mean_squared_error': 0.14721714875954123, 'root_relative_squared_error': 0.3122947325844614, 'total_cost': 0.0, 'unweighted_recall': 0.9533333333333333}


In [2]:
def get_algorithm_ranking_for_dataset(dataset_id, metric="predictive_accuracy"):

    tasks = openml.tasks.list_tasks(
        data_id=dataset_id,
        task_type=TaskType.SUPERVISED_CLASSIFICATION,
        output_format="dataframe"
    )
    if tasks.empty:
        return None

    runs = openml.runs.list_runs(
        task=tasks.index.tolist(),
        output_format="dataframe"
    )
    if runs.empty:
        return None

    rows = []

    for run_id, row in runs.iterrows():
        try:
            run = openml.runs.get_run(run_id)
            score = run.evaluations.get(metric)
            if score is None:
                continue
            rows.append({
                "algorithm": row["flow_name"],
                "score": score
            })
        except:
            continue

    if not rows:
        return None

    df = pd.DataFrame(rows)
    agg = df.groupby("algorithm")["score"].max().reset_index()

    ranking = agg.sort_values("score", ascending=False).reset_index(drop=True)
    ranking["rank"] = ranking.index + 1
    ranking["metric"] = metric

    return ranking




In [3]:
ranking = get_algorithm_ranking_for_dataset(1)

if ranking is None:
    print("No hay runs con métricas válidas")
else:
    print(ranking.head())


# dataset_id   nombre
# -----------  -------------------------
# 3            kr-vs-kp
# 6            letter
# 11           balance-scale
# 12           mfeat-factors
# 15           breast-w
# 31           credit-g
# 37           diabetes
# 44           spambase
# 50           tic-tac-toe



No hay runs con métricas válidas
